# **04 MODELING (NESTED CROSS-VALIDATION)**

In [2]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  

from scipy.stats import randint, uniform, loguniform
from statsmodels.tsa.seasonal import seasonal_decompose
from category_encoders import *


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import (
    train_test_split,
    cross_validate,
    RandomizedSearchCV,
    StratifiedKFold,
    KFold,
    cross_val_score
)
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer,
)
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
    ExtraTreesRegressor
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
)
from sklearn.dummy import DummyRegressor

# ——— Additional models ————————————————————————
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')



In [2]:


# Project root & data paths
project_root = Path().resolve().parent
df_pre_path = project_root / "data" / "interim" / "data_preprocessed.parquet"
df = pd.read_parquet(df_pre_path)

src_path = project_root / "src"
sys.path.append(str(src_path))


First thing, I remove outliers in the target variable price, and I log-transform it.

In [3]:

# Remove outliers in price - although I guess that if I had to put in production, it would be more right to remove outliers only in the training set. 

Q1 = df["price"].quantile(0.25)
Q3 = df["price"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df["price"] >= lower_bound) & (df["price"] <= upper_bound)]

# Target transformation and feature preparation
y = np.log1p(df["price"])
X = df.drop(columns=["price"])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

I now recall the features in their respective categories in order to be able to feed them to the pipeline, recalling what I did in the previous notebooks.

In [4]:


# 4. Define feature groups
numeric_features = [
    'host_listings_count', 'host_total_listings_count',
    'accommodates', 'bathrooms', 'bedrooms', 'beds',
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
    'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'calculated_host_listings_count',
    'calculated_host_listings_count_private_rooms',
    'reviews_per_month', 'days_since_host_since',
    'air_conditioning', 'elevator', 'fast_wifi', 'parking',
    'coffee_machine', 'washer', 'self_check_in', 'streaming_tv',
    'dedicated_workspace', 'private_entrance', 'kitchen_appliances',
    'heating', 'hot_water', 'safety_equipment', 'clothing_storage',
    'balcony', 'premium_views', 'dishwasher', 'gym'
]

categorical_features = [
    'neighbourhood_cleansed',
    'property_type',
    'room_type',
    'host_location'
]

ordinal_features_days = [
    'days_since_first_review',
    'days_since_last_review'
]

# I will keep just one since they are very correlated
ordinal_features_reviews = [
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value']

ordinal_features = ordinal_features_days + ordinal_features_reviews

# 5. Define category orders for OrdinalEncoder
first_review_order = [
    'no_review_yet',
    'very_new (<= 1 month)',
    'new (<= 6 months)',
    'established (<= 1 year)',
    'mature (<= 3 years)',
    'veteran (<= 5 years)',
    'legacy (over 5 years)'
]

last_review_order = [
    'no_review',
    'very_recent (<= 1 week)',
    'recent (<= 1 month)',
    'somewhat_recent (<= 3 months)',
    'old (<= 6 months)',
    'very_old (<= 1 year)',
    'dormant (over a year)'
]


review_order = ["no_reviews", "low_reviews", "medium_reviews", "high_reviews", "top_reviews"]

all_ord_categories = (
    [first_review_order, last_review_order] +
    [review_order] * len(ordinal_features_reviews)
)



In [5]:
# 6. Build transformers
numeric_transformer = Pipeline([
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

ordinal_transformer = Pipeline([
    ("ordinal", OrdinalEncoder(categories=all_ord_categories))
])

# 7. Combine into ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer,   numeric_features),
    ("cat", categorical_transformer, categorical_features),
    ("ord", ordinal_transformer,    ordinal_features),
], remainder="drop")

# 8. Create full Pipeline
model_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("select", SelectKBest(k=50)),
    ("regressor", RandomForestRegressor(random_state=42))
])

In [6]:
models_and_params = {

    
    'OLS': {
        'model': LinearRegression(),
        'params': {}  # No hyperparameters to tune
    },
    
    'Ridge': {
        'model': Ridge(random_state=42),
        'params': {
            'regressor__alpha': loguniform(0.01, 100),
            'select__k': randint(20, 100)
        }
    },
    
    'ElasticNet': {
        'model': ElasticNet(random_state=42),
        'params': {
            'regressor__alpha': loguniform(0.01, 10),
            'regressor__l1_ratio': uniform(0.1, 0.8),
            'select__k': randint(20, 100)
        }
    },
    
    'RandomForest': {
        'model': RandomForestRegressor(n_jobs=-1, random_state=42),
        'params': {
            'regressor__n_estimators': randint(100, 500),
            'regressor__max_depth': randint(5, 20),
            'regressor__min_samples_split': randint(2, 10),
            'regressor__min_samples_leaf': randint(1, 5),
            'select__k': randint(30, 120)
        }
    },
    
    'ExtraTrees': {
        'model': ExtraTreesRegressor(n_jobs=-1, random_state=42),
        'params': {
            'regressor__n_estimators': randint(100, 500),
            'regressor__max_depth': randint(5, 20),
            'regressor__min_samples_split': randint(2, 10),
            'regressor__min_samples_leaf': randint(1, 5),
            'select__k': randint(30, 120)
        }
    },
    
    'HistGB': {
        'model': HistGradientBoostingRegressor(random_state=42),
        'params': {
            'regressor__max_depth': randint(3, 10),
            'regressor__learning_rate': loguniform(0.01, 0.3),
            'regressor__max_iter': randint(100, 500),
            'regressor__l2_regularization': loguniform(1e-4, 10),
            'select__k': randint(30, 120)
        }
    },
    
    'GradientBoosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'regressor__n_estimators': randint(100, 500),
            'regressor__learning_rate': loguniform(0.01, 0.3),
            'regressor__max_depth': randint(3, 10),
            'regressor__subsample': uniform(0.6, 0.4),
            'select__k': randint(30, 120)
        }
    },
    
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {
            'regressor__n_neighbors': randint(3, 20),
            'regressor__weights': ['uniform', 'distance'],
            'regressor__p': [1, 2],
            'select__k': randint(20, 80)
        }
    },
    
    'SVR': {
        'model': SVR(),
        'params': {
            'regressor__C': loguniform(0.1, 100),
            'regressor__epsilon': loguniform(0.01, 1),
            'regressor__kernel': ['rbf', 'linear'],
            'select__k': randint(20, 80)
        }
    },
    
    
    'XGBoost': {
        'model': XGBRegressor(objective="reg:squarederror", n_jobs=-1, random_state=42),
        'params': {
            'regressor__n_estimators': randint(200, 1000),
            'regressor__max_depth': randint(4, 12),
            'regressor__learning_rate': loguniform(0.01, 0.3),
            'regressor__subsample': uniform(0.6, 0.4),
            'regressor__colsample_bytree': uniform(0.6, 0.4),
            'regressor__gamma': loguniform(1e-8, 1e-1),
            'select__k': randint(30, 120)
        }
    },
    
    'CatBoost': {
        'model': CatBoostRegressor(
            loss_function="RMSE",
            random_seed=42,
            verbose=0,
            allow_writing_files=False
        ),
        'params': {
            'regressor__iterations': randint(200, 1000),
            'regressor__depth': randint(4, 10),
            'regressor__learning_rate': loguniform(0.01, 0.3),
            'regressor__l2_leaf_reg': loguniform(1, 10),
            'select__k': randint(30, 120)
        }
    }
}

### I will run a Nested Cross Validation since I want to compare different models with different parameters. 

In [7]:
def nested_cross_validation_regression(X, y, models_and_params, outer_cv=5, inner_cv=3, 
                                     n_iter=20, scoring='r2', random_state=42):
    """
    Perform nested cross-validation for regression model selection and performance estimation.
    
    Parameters:
    -----------
    X : array-like, shape (n_samples, n_features)
        Feature matrix
    y : array-like, shape (n_samples,)
        Target vector
    models_and_params : dict
        Dictionary containing models and their hyperparameter grids
    outer_cv : int
        Number of folds for outer cross-validation
    inner_cv : int
        Number of folds for inner cross-validation (hyperparameter tuning)
    n_iter : int
        Number of parameter settings sampled for RandomizedSearchCV
    scoring : str
        Scoring metric
    random_state : int
        Random state for reproducibility
    
    Returns:
    --------
    results : dict
        Dictionary containing results for each model
    """
    
    # Initialize cross-validation splitters
    outer_cv_splitter = KFold(n_splits=outer_cv, shuffle=True, random_state=random_state)
    inner_cv_splitter = KFold(n_splits=inner_cv, shuffle=True, random_state=random_state)
    
    results = {}
    
    print("=== NESTED CROSS-VALIDATION FOR AIRBNB PRICE PREDICTION ===\n")
    print(f"Dataset shape: {X.shape}")
    print(f"Target range: {y.min():.3f} - {y.max():.3f} (log-transformed)")
    print(f"Outer CV: {outer_cv} folds, Inner CV: {inner_cv} folds")
    print(f"Hyperparameter search iterations: {n_iter}\n")
    
    for model_name, model_config in models_and_params.items():
        print(f"Evaluating {model_name}...")
        
        # Create pipeline with preprocessing, feature selection, and classifier
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('select', SelectKBest(k=50)),  # Default value, will be tuned if in params
            ('regressor', model_config['model'])
        ])
        
        # Outer loop: Performance estimation
        outer_scores = []
        best_params_per_fold = []
        
        fold = 1
        for train_idx, test_idx in outer_cv_splitter.split(X):
            print(f"  Processing outer fold {fold}/{outer_cv}")
            
            X_train_outer, X_test_outer = X.iloc[train_idx], X.iloc[test_idx]
            y_train_outer, y_test_outer = y.iloc[train_idx], y.iloc[test_idx]
            
            # Inner loop: Hyperparameter optimization
            if model_config['params']:  # Only tune if there are parameters to tune
                search = RandomizedSearchCV(
                    pipeline, 
                    model_config['params'],
                    cv=inner_cv_splitter,
                    scoring=scoring,
                    n_iter=n_iter,
                    n_jobs=-1,
                    random_state=random_state,
                    return_train_score=False
                )
                
                # Fit grid search on outer training set
                search.fit(X_train_outer, y_train_outer)
                
                # Get best model from inner CV
                best_model = search.best_estimator_
                best_params_per_fold.append(search.best_params_)
            else:
                # No hyperparameters to tune, just fit the pipeline
                pipeline.fit(X_train_outer, y_train_outer)
                best_model = pipeline
                best_params_per_fold.append({})
            
            # Evaluate best model on outer test set
            y_pred = best_model.predict(X_test_outer)
            if scoring == 'r2':
                fold_score = r2_score(y_test_outer, y_pred)
            elif scoring == 'neg_mean_squared_error':
                fold_score = -mean_squared_error(y_test_outer, y_pred)
            elif scoring == 'neg_mean_absolute_error':
                fold_score = -mean_absolute_error(y_test_outer, y_pred)
            else:
                fold_score = best_model.score(X_test_outer, y_test_outer)
            
            outer_scores.append(fold_score)
            fold += 1
        
        # Store results
        results[model_name] = {
            'outer_scores': outer_scores,
            'mean_score': np.mean(outer_scores),
            'std_score': np.std(outer_scores),
            'best_params_per_fold': best_params_per_fold
        }
        
        print(f"  Mean {scoring}: {np.mean(outer_scores):.4f} (+/- {np.std(outer_scores):.4f})")
        print(f"  Individual fold scores: {[f'{score:.4f}' for score in outer_scores]}")
        print()
    
    return results

def select_best_model_and_retrain(X, y, models_and_params, results, inner_cv=3, n_iter=50):
    """
    Select the best model based on nested CV results and retrain on full dataset.
    """
    # Find best model
    best_model_name = max(results.keys(), key=lambda k: results[k]['mean_score'])
    best_model_config = models_and_params[best_model_name]
    
    print(f"=== BEST MODEL SELECTION ===")
    print(f"Best model: {best_model_name}")
    print(f"Expected performance: {results[best_model_name]['mean_score']:.4f} "
          f"(+/- {results[best_model_name]['std_score']:.4f})")
    print()
    
    # Retrain best model on full dataset with hyperparameter tuning
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('select', SelectKBest(k=50)),
        ('regressor', best_model_config['model'])
    ])
    
    if best_model_config['params']:
        inner_cv_splitter = KFold(n_splits=inner_cv, shuffle=True, random_state=42)
        
        final_search = RandomizedSearchCV(
            pipeline,
            best_model_config['params'],
            cv=inner_cv_splitter,
            scoring='r2',
            n_iter=n_iter,
            n_jobs=-1,
            random_state=42
        )
        
        final_search.fit(X, y)
        final_model = final_search.best_estimator_
        
        print(f"Final model hyperparameters: {final_search.best_params_}")
        print(f"Cross-validation score on full dataset: {final_search.best_score_:.4f}")
    else:
        pipeline.fit(X, y)
        final_model = pipeline
        print("No hyperparameters to tune for this model.")
    
    return final_model, best_model_name


In [8]:

print("Starting Nested Cross-Validation for Airbnb Price Prediction...")
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print()

# Perform nested cross-validation (reduced subset for demonstration)
# You can include all models by using the full models_and_params dictionary
selected_models = models_and_params


cv_results = nested_cross_validation_regression(
    X_train, y_train, 
    selected_models,  # Use selected_models or models_and_params for all
    outer_cv=5, 
    inner_cv=3, 
    n_iter=20,  # Reduced for faster execution
    scoring='r2'
)

# Select and retrain best model
final_model, best_model_name = select_best_model_and_retrain(
    X_train, y_train, selected_models, cv_results, n_iter=30
)



Starting Nested Cross-Validation for Airbnb Price Prediction...
Training set shape: (16176, 53)
Test set shape: (4045, 53)

=== NESTED CROSS-VALIDATION FOR AIRBNB PRICE PREDICTION ===

Dataset shape: (16176, 53)
Target range: 2.708 - 5.897 (log-transformed)
Outer CV: 5 folds, Inner CV: 3 folds
Hyperparameter search iterations: 20

Evaluating OLS...
  Processing outer fold 1/5
  Processing outer fold 2/5
  Processing outer fold 3/5
  Processing outer fold 4/5
  Processing outer fold 5/5
  Mean r2: 0.3862 (+/- 0.0158)
  Individual fold scores: ['0.3759', '0.3875', '0.3971', '0.3627', '0.4078']

Evaluating Ridge...
  Processing outer fold 1/5
  Processing outer fold 2/5
  Processing outer fold 3/5
  Processing outer fold 4/5
  Processing outer fold 5/5
  Mean r2: 0.4163 (+/- 0.0191)
  Individual fold scores: ['0.4011', '0.4112', '0.4302', '0.3935', '0.4455']

Evaluating ElasticNet...
  Processing outer fold 1/5
  Processing outer fold 2/5
  Processing outer fold 3/5
  Processing outer fol

## FINAL EVALUATION ON TEST SET

In [9]:


print("\n=== FINAL TEST SET EVALUATION ===")
y_pred_test = final_model.predict(X_test)

test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"Test R²: {test_r2:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test MAE: {test_mae:.4f}")

# Convert back from log scale for interpretation
y_test_original = np.expm1(y_test)
y_pred_original = np.expm1(y_pred_test)

test_rmse_original = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
test_mae_original = mean_absolute_error(y_test_original, y_pred_original)

print(f"\nIn original price scale:")
print(f"Test RMSE: ${test_rmse_original:.2f}")
print(f"Test MAE: ${test_mae_original:.2f}")

# Summary of all models
print("\n=== NESTED CV SUMMARY ===")
cv_summary = pd.DataFrame([
    {
        'Model': model_name,
        'Mean_R2': result['mean_score'],
        'Std_R2': result['std_score'],
        'CI_Lower': result['mean_score'] - 1.96 * result['std_score'],
        'CI_Upper': result['mean_score'] + 1.96 * result['std_score']
    }
    for model_name, result in cv_results.items()
]).sort_values('Mean_R2', ascending=False)

print(cv_summary.round(4))

print(f"\nSelected model: {best_model_name}")
print("Note: The nested CV performance estimates represent unbiased estimates")
print("of how well each model is expected to perform on unseen data.")


=== FINAL TEST SET EVALUATION ===
Test R²: 0.5870
Test RMSE: 0.3230
Test MAE: 0.2468

In original price scale:
Test RMSE: $49.26
Test MAE: $34.51

=== NESTED CV SUMMARY ===
               Model  Mean_R2  Std_R2  CI_Lower  CI_Upper
9            XGBoost   0.5639  0.0151    0.5344    0.5935
10          CatBoost   0.5567  0.0162    0.5250    0.5884
6   GradientBoosting   0.5444  0.0161    0.5129    0.5759
5             HistGB   0.5422  0.0158    0.5112    0.5733
3       RandomForest   0.5002  0.0175    0.4660    0.5345
4         ExtraTrees   0.4911  0.0164    0.4589    0.5234
8                SVR   0.4589  0.0139    0.4316    0.4862
1              Ridge   0.4163  0.0191    0.3789    0.4537
0                OLS   0.3862  0.0158    0.3553    0.4171
2         ElasticNet   0.3655  0.0180    0.3302    0.4009
7                KNN   0.3621  0.0073    0.3478    0.3765

Selected model: XGBoost
Note: The nested CV performance estimates represent unbiased estimates
of how well each model is expected